In [ ]:
import pandas as pd
import numpy as np

# **Base Code**

In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = []
k=len(df_smiles)
for s in df_smiles['SMILES'][:k]:
    response=openai.Embedding.create(
      model="text-embedding-ada-002",
      input=s)
    SMILES_Embedding = [e["embedding"] for e in response["data"]]
    embeddings.extend(SMILES_Embedding)

matrix = np.array(embeddings)
df_smiles=pd.concat([df_smiles, pd.DataFrame(matrix)], axis=1)

In [ ]:
df_valid_molecules  = pd.read_csv('/content/drive/MyDrive/VRSEC/III Year/3 - 2/Mini/Teju/df_valid_molecules.csv')
drugID_DESC = pd.read_csv('/content/drive/MyDrive/VRSEC/III Year/3 - 2/Mini/Teju/drugID_DESC.csv')
DDI_graph = pd.read_csv('/content/drive/MyDrive/VRSEC/III Year/3 - 2/Mini/Teju/DDI_graph.csv')
drugID_smiles_ddi = pd.read_csv('/content/drive/MyDrive/VRSEC/III Year/3 - 2/Mini/Teju/drugID_smiles_ddi.csv')

# Embedding Model

# **T5 Model**

In [ ]:
pip install transformers datasets pandas rdkit torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch
import numpy as np

In [ ]:
# Prepare HuggingFace Dataset
dataset = Dataset.from_pandas(df_smiles[['DrugBank ID', 'SMILES']])

In [ ]:
# Load T5 tokenizer and model (ChemT5 or vanilla T5)
model_name = "t5-base"  # or use a chem-pretrained version like 'chitholian/chemT5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
    inputs = tokenizer(example["SMILES"], truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["SMILES"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
# Prepare training arguments
training_args = TrainingArguments(
    output_dir="./t5-smiles-finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_strategy="epoch",
    logging_steps=100,
    eval_strategy="no",
    learning_rate=3e-4,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
)

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/11583 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Data collator for T5 (denoising autoencoding)
# This collator will automatically create decoder_input_ids from the 'labels'
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

<ipython-input-19-337e38228f98>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,0.589400
200,0.013100
300,0.007300
400,0.008300
500,0.005600
600,0.004300
700,0.003600
800,0.003800
900,0.004100
1000,0.003100


TrainOutput(global_step=3620, training_loss=0.018622563181969837, metrics={'train_runtime': 1666.6012, 'train_samples_per_second': 34.75, 'train_steps_per_second': 2.172, 'total_flos': 8816949057945600.0, 'train_loss': 0.018622563181969837, 'epoch': 5.0})

In [ ]:
trainer.save_model("./t5-smiles-final")
tokenizer.save_pretrained("./t5-smiles-final")

('./t5-smiles-final/tokenizer_config.json',
 './t5-smiles-final/special_tokens_map.json',
 './t5-smiles-final/spiece.model',
 './t5-smiles-final/added_tokens.json')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("./t5-smiles-final")
model = T5ForConditionalGeneration.from_pretrained("./t5-smiles-final")

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load fine-tuned model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("/content/t5-smiles-final")
model = T5ForConditionalGeneration.from_pretrained("/content/t5-smiles-final")
model.eval()

embeddings = []

# Add tqdm for visibility
with torch.no_grad():
    for smiles in tqdm(df_smiles["SMILES"], desc="Generating embeddings"):
        inputs = tokenizer(smiles, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoder_output = model.encoder(**inputs).last_hidden_state
        mean_embedding = encoder_output.mean(dim=1).squeeze().numpy()  # Mean Pooling
        embeddings.append(mean_embedding)

# Stack into a matrix and join with original DataFrame
matrix = np.vstack(embeddings)
df_result = pd.concat([df_smiles, pd.DataFrame(matrix)], axis=1)


Generating embeddings: 100%|██████████| 11583/11583 [1:30:32<00:00,  2.13it/s]


In [ ]:
df_result

,DrugBank ID,SMILES,0,1,2,3,4,5,6,7,...,758,759,760,761,762,763,764,765,766,767
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,-0.021165,-0.085178,-0.050315,-0.008298,0.025406,-0.184038,0.090913,-0.079128,...,0.004342,0.017825,-0.075881,-0.099598,-0.011201,0.069399,0.147426,-0.115494,-0.038739,0.191575
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,-0.022240,-0.108156,-0.032091,-0.006497,0.050381,-0.156554,0.106840,-0.069328,...,0.005188,0.005579,-0.088282,-0.078788,-0.012340,0.098765,0.157499,-0.073734,-0.034575,0.184224
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,-0.031897,-0.085769,-0.040033,0.006516,0.030664,-0.153603,0.117109,-0.064343,...,0.004844,0.016106,-0.090566,-0.080296,-0.027231,0.115448,0.174454,-0.110197,-0.021406,0.155667
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,-0.063853,-0.121722,-0.037007,0.024461,0.052812,-0.141032,0.099662,-0.075215,...,-0.004149,0.018953,-0.122422,-0.115590,0.013338,0.087100,0.128433,-0.049607,-0.032828,0.220184
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,-0.052248,-0.055574,-0.052846,0.002127,0.027879,-0.142792,0.108183,-0.071169,...,0.021194,-0.001967,-0.087060,-0.062268,-0.021073,0.089268,0.191457,-0.142776,-0.030837,0.146333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11578,DB17379,CC(C)C1=C(O)C(O)=C(C=O)C2=C(O)C(=C(C)C=C12)C1=...,0.008380,-0.065302,-0.056838,0.037103,0.029115,-0.085376,0.060096,-0.093109,...,-0.107091,0.094395,-0.111567,0.014821,-0.014915,0.097573,0.192382,0.009118,0.013016,0.031876
11579,DB17383,CN1CCN(CC2=CC=C(NC(=O)C3=NNC=C3NC3=C4C=CNC4=NC...,0.013242,-0.230412,0.024410,0.061824,0.020288,-0.042336,0.124764,0.030553,...,-0.246780,0.189671,0.012922,0.050104,-0.170736,0.090722,0.174056,0.113921,0.076842,-0.029377
11580,DB17384,CC1=C2N=C(C3=CC=CC=C3Cl)C3=C(NC2=NN1)C=C(N=C3)...,0.049051,-0.145739,0.027875,0.086474,0.019988,0.002158,0.091650,0.056862,...,-0.293736,0.169267,-0.013733,0.079881,-0.090430,0.095455,0.227601,0.112589,0.039581,-0.051948
11581,DB17385,CC[C@@]1(OC(=O)C(C)ON=C2C3=C(C4=C2C=C(C=C4[N+]...,-0.017617,0.029630,-0.093336,0.007737,-0.005551,-0.178476,0.096762,-0.071214,...,0.006572,0.017730,-0.078023,-0.010486,0.001671,0.064636,0.189312,-0.083924,-0.040092,0.161461


In [ ]:
# prompt: save the df_result into csv
df_result.to_csv("T5_SMILES_Embeddings.csv", index=False)